# Prisoner Dilemma Experiment on Stock Data

This notebook implements a prisoner dilemma simulation using stock market data. The experiment explores how different strategies perform when applied to stock price movements.

## Strategy Representation

Each strategy is represented by a 4-bit binary pattern that determines the action based on:
- Opponent's last two moves (CC, CD, DC, DD)
- Opponent's last move (C or D)

## Mapping Rules
- Price up → Cooperate (C)
- Price down → Defect (D)
- Buy → Cooperate (C)
- Sell → Defect (D)

In [ ]:
# Install required packages
!pip install kagglehub pandas numpy matplotlib seaborn -q

In [ ]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Tuple

# Set style for better visualizations
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)

## 1. Load Stock Data from Kaggle

In [ ]:
# Load the dataset
file_path = "indexData.csv"

df = kagglehub.dataset_load(
    KaggleDatasetAdapter.PANDAS,
    "mattiuzc/stock-exchange-data",
    file_path,
)

print("First 5 records:")
print(df.head())
print("\nColumns:")
print(df.columns)
print("\nDataset shape:", df.shape)

## 2. Data Preprocessing

Convert stock price movements to cooperate/defect actions.

In [ ]:
# Prepare the data
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values('Date').reset_index(drop=True)

# Calculate price changes
df['Price_Change'] = df['Adj Close'].diff()
df['Price_Up'] = df['Price_Change'] > 0

# Map to Cooperate (C=0) or Defect (D=1)
# Price up → Cooperate (0), Price down → Defect (1)
df['Market_Action'] = (~df['Price_Up']).astype(int)

# Remove NaN values from first row
df = df.dropna().reset_index(drop=True)

print("Processed data:")
print(df.head(10))
print(f"\nTotal trading days: {len(df)}")
print(f"Cooperate days (price up): {(df['Market_Action'] == 0).sum()}")
print(f"Defect days (price down): {(df['Market_Action'] == 1).sum()}")

## 3. Define All 16 Possible Strategies

Each strategy is a 4-bit pattern (0000 to 1111) that determines the action based on:
- Bit 0: Action when opponent's last two moves are CC and last move is C
- Bit 1: Action when opponent's last two moves are CD and last move is D
- Bit 2: Action when opponent's last two moves are DC and last move is C
- Bit 3: Action when opponent's last two moves are DD and last move is D

In [ ]:
class Strategy:
    """Represents a prisoner dilemma strategy based on opponent's history."""
    
    def __init__(self, strategy_bits: int):
        """
        Initialize strategy with a 4-bit pattern.
        
        Args:
            strategy_bits: Integer from 0-15 representing the 4-bit strategy
        """
        self.bits = strategy_bits
        self.binary = format(strategy_bits, '04b')
        self.name = f"Strategy_{self.binary}"
        
    def get_action(self, opponent_last_two: Tuple[int, int], opponent_last: int) -> int:
        """
        Determine action based on opponent's history.
        
        Args:
            opponent_last_two: Tuple of opponent's last two moves (e.g., (0, 0) for CC)
            opponent_last: Opponent's last move (0=C, 1=D)
            
        Returns:
            0 for Cooperate, 1 for Defect
        """
        # Determine which bit to use based on opponent's history
        if opponent_last_two == (0, 0) and opponent_last == 0:  # CC, C
            bit_index = 0
        elif opponent_last_two == (0, 1) and opponent_last == 1:  # CD, D
            bit_index = 1
        elif opponent_last_two == (1, 0) and opponent_last == 0:  # DC, C
            bit_index = 2
        elif opponent_last_two == (1, 1) and opponent_last == 1:  # DD, D
            bit_index = 3
        else:
            # Default to cooperate if pattern doesn't match
            return 0
            
        # Extract the bit at the specified index
        return (self.bits >> bit_index) & 1
    
    def __repr__(self):
        return f"{self.name}"


# Create all 16 possible strategies
all_strategies = [Strategy(i) for i in range(16)]

print("All 16 Possible Strategies:")
print("-" * 50)
for i, strategy in enumerate(all_strategies):
    print(f"{i:2d}. {strategy.name} (Binary: {strategy.binary}, Decimal: {strategy.bits})")

print("\nStrategy Interpretation:")
print("Each bit represents the action (0=Cooperate, 1=Defect) for:")
print("  Bit 0: Opponent's last two moves = CC, last move = C")
print("  Bit 1: Opponent's last two moves = CD, last move = D")
print("  Bit 2: Opponent's last two moves = DC, last move = C")
print("  Bit 3: Opponent's last two moves = DD, last move = D")

## 4. Simulate Strategies

Simulate each strategy playing against the market (stock price movements).

In [ ]:
def calculate_payoff(player_action: int, opponent_action: int) -> Tuple[float, float]:
    """
    Calculate payoffs for both players based on their actions.
    
    Classic Prisoner's Dilemma payoff matrix:
    - Both Cooperate: (3, 3)
    - Player Cooperates, Opponent Defects: (0, 5)
    - Player Defects, Opponent Cooperates: (5, 0)
    - Both Defect: (1, 1)
    
    Args:
        player_action: 0 for Cooperate, 1 for Defect
        opponent_action: 0 for Cooperate, 1 for Defect
        
    Returns:
        Tuple of (player_payoff, opponent_payoff)
    """
    payoff_matrix = {
        (0, 0): (3, 3),  # Both cooperate
        (0, 1): (0, 5),  # Player cooperates, opponent defects
        (1, 0): (5, 0),  # Player defects, opponent cooperates
        (1, 1): (1, 1),  # Both defect
    }
    return payoff_matrix[(player_action, opponent_action)]


def simulate_strategy(strategy: Strategy, market_actions: List[int], 
                     initial_capital: float = 10000.0) -> dict:
    """
    Simulate a strategy against market movements.
    
    Args:
        strategy: Strategy object to simulate
        market_actions: List of market actions (0=Cooperate/up, 1=Defect/down)
        initial_capital: Starting capital
        
    Returns:
        Dictionary containing simulation results
    """
    total_payoff = 0
    capital = initial_capital
    history = []
    player_actions = []
    
    # Start with default cooperate actions
    player_history = [0, 0]  # Initialize with two cooperates
    market_history = market_actions[:2] if len(market_actions) >= 2 else [0, 0]
    
    for i in range(2, len(market_actions)):
        market_action = market_actions[i]
        
        # Get opponent's (market's) last two moves and last move
        opponent_last_two = (market_history[-2], market_history[-1])
        opponent_last = market_history[-1]
        
        # Get player action from strategy
        player_action = strategy.get_action(opponent_last_two, opponent_last)
        
        # Calculate payoffs
        player_payoff, market_payoff = calculate_payoff(player_action, market_action)
        total_payoff += player_payoff
        
        # Update capital (simplified: payoff represents percentage change)
        capital += player_payoff * 10  # Scale factor for visualization
        
        # Record history
        history.append({
            'round': i,
            'player_action': player_action,
            'market_action': market_action,
            'payoff': player_payoff,
            'total_payoff': total_payoff,
            'capital': capital
        })
        
        player_actions.append(player_action)
        
        # Update histories
        market_history.append(market_action)
        player_history.append(player_action)
    
    cooperate_count = sum(1 for a in player_actions if a == 0)
    defect_count = len(player_actions) - cooperate_count
    
    return {
        'strategy': strategy,
        'total_payoff': total_payoff,
        'final_capital': capital,
        'yield': (capital - initial_capital) / initial_capital * 100,
        'avg_payoff': total_payoff / len(player_actions) if player_actions else 0,
        'cooperate_count': cooperate_count,
        'defect_count': defect_count,
        'cooperate_rate': cooperate_count / len(player_actions) if player_actions else 0,
        'history': history
    }


print("Starting simulation of all strategies...")
market_actions = df['Market_Action'].tolist()

results = []
for strategy in all_strategies:
    result = simulate_strategy(strategy, market_actions)
    results.append(result)
    print(f"Completed: {strategy.name}")

print("\nSimulation completed!")

## 5. Analyze and Display Yields

Show the performance of all strategies.

In [ ]:
# Create summary dataframe
summary_data = []
for result in results:
    summary_data.append({
        'Strategy': result['strategy'].name,
        'Binary': result['strategy'].binary,
        'Total Payoff': result['total_payoff'],
        'Final Capital': result['final_capital'],
        'Yield (%)': result['yield'],
        'Avg Payoff': result['avg_payoff'],
        'Cooperate Rate': result['cooperate_rate'],
        'Cooperate Count': result['cooperate_count'],
        'Defect Count': result['defect_count']
    })

summary_df = pd.DataFrame(summary_data)
summary_df = summary_df.sort_values('Yield (%)', ascending=False)

print("="*100)
print("PERFORMANCE SUMMARY OF ALL STRATEGIES")
print("="*100)
print(summary_df.to_string(index=False))
print("\n" + "="*100)

In [ ]:
# Display top 5 and bottom 5 strategies
print("\n" + "="*80)
print("TOP 5 STRATEGIES BY YIELD")
print("="*80)
print(summary_df.head(5).to_string(index=False))

print("\n" + "="*80)
print("BOTTOM 5 STRATEGIES BY YIELD")
print("="*80)
print(summary_df.tail(5).to_string(index=False))

## 6. Visualizations

In [ ]:
# Plot 1: Yields comparison
plt.figure(figsize=(14, 8))
colors = ['green' if y > 0 else 'red' for y in summary_df['Yield (%)']]
plt.barh(summary_df['Strategy'], summary_df['Yield (%)'], color=colors)
plt.xlabel('Yield (%)', fontsize=12)
plt.ylabel('Strategy', fontsize=12)
plt.title('Yield Performance of All 16 Strategies', fontsize=14, fontweight='bold')
plt.axvline(x=0, color='black', linestyle='--', linewidth=0.8)
plt.tight_layout()
plt.show()

In [ ]:
# Plot 2: Total Payoff comparison
plt.figure(figsize=(14, 8))
sorted_by_payoff = summary_df.sort_values('Total Payoff', ascending=True)
colors = ['green' if p > 0 else 'red' for p in sorted_by_payoff['Total Payoff']]
plt.barh(sorted_by_payoff['Strategy'], sorted_by_payoff['Total Payoff'], color=colors)
plt.xlabel('Total Payoff', fontsize=12)
plt.ylabel('Strategy', fontsize=12)
plt.title('Total Payoff of All 16 Strategies', fontsize=14, fontweight='bold')
plt.axvline(x=0, color='black', linestyle='--', linewidth=0.8)
plt.tight_layout()
plt.show()

In [ ]:
# Plot 3: Cooperate Rate vs Yield
plt.figure(figsize=(12, 8))
scatter = plt.scatter(summary_df['Cooperate Rate'], summary_df['Yield (%)'], 
                     s=200, alpha=0.6, c=summary_df['Total Payoff'], 
                     cmap='viridis', edgecolors='black', linewidth=1.5)

# Add labels for each point
for idx, row in summary_df.iterrows():
    plt.annotate(row['Binary'], 
                (row['Cooperate Rate'], row['Yield (%)']),
                fontsize=8, ha='center', va='center')

plt.xlabel('Cooperation Rate', fontsize=12)
plt.ylabel('Yield (%)', fontsize=12)
plt.title('Cooperation Rate vs Yield Performance', fontsize=14, fontweight='bold')
plt.colorbar(scatter, label='Total Payoff')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Plot 4: Capital evolution for top 5 strategies
plt.figure(figsize=(14, 8))

top_5_strategies = summary_df.head(5)['Strategy'].tolist()
for result in results:
    if result['strategy'].name in top_5_strategies:
        history_df = pd.DataFrame(result['history'])
        plt.plot(history_df['round'], history_df['capital'], 
                label=result['strategy'].name, linewidth=2, alpha=0.8)

plt.xlabel('Trading Day', fontsize=12)
plt.ylabel('Capital ($)', fontsize=12)
plt.title('Capital Evolution - Top 5 Strategies', fontsize=14, fontweight='bold')
plt.legend(loc='best', fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Plot 5: Heatmap of strategy performance metrics
plt.figure(figsize=(12, 10))

# Prepare data for heatmap
heatmap_data = summary_df[['Strategy', 'Total Payoff', 'Yield (%)', 
                           'Avg Payoff', 'Cooperate Rate']].set_index('Strategy')

# Normalize for better visualization
heatmap_normalized = (heatmap_data - heatmap_data.min()) / (heatmap_data.max() - heatmap_data.min())

sns.heatmap(heatmap_normalized, annot=False, cmap='RdYlGn', 
            cbar_kws={'label': 'Normalized Value'}, linewidths=0.5)
plt.title('Strategy Performance Heatmap (Normalized)', fontsize=14, fontweight='bold')
plt.xlabel('Metrics', fontsize=12)
plt.ylabel('Strategy', fontsize=12)
plt.tight_layout()
plt.show()

## 7. Statistical Analysis

In [ ]:
print("\n" + "="*80)
print("STATISTICAL SUMMARY")
print("="*80)
print("\nYield Statistics:")
print(f"  Mean Yield: {summary_df['Yield (%)'].mean():.2f}%")
print(f"  Median Yield: {summary_df['Yield (%)'].median():.2f}%")
print(f"  Std Dev: {summary_df['Yield (%)'].std():.2f}%")
print(f"  Min Yield: {summary_df['Yield (%)'].min():.2f}%")
print(f"  Max Yield: {summary_df['Yield (%)'].max():.2f}%")

print("\nCooperation Statistics:")
print(f"  Mean Cooperation Rate: {summary_df['Cooperate Rate'].mean():.2%}")
print(f"  Median Cooperation Rate: {summary_df['Cooperate Rate'].median():.2%}")

print("\nPayoff Statistics:")
print(f"  Mean Total Payoff: {summary_df['Total Payoff'].mean():.2f}")
print(f"  Median Total Payoff: {summary_df['Total Payoff'].median():.2f}")
print(f"  Mean Avg Payoff: {summary_df['Avg Payoff'].mean():.2f}")

print("\n" + "="*80)

## 8. Insights and Conclusions

In [ ]:
best_strategy = summary_df.iloc[0]
worst_strategy = summary_df.iloc[-1]

print("\n" + "="*80)
print("KEY INSIGHTS")
print("="*80)

print(f"\n1. BEST PERFORMING STRATEGY:")
print(f"   - Strategy: {best_strategy['Strategy']} (Binary: {best_strategy['Binary']})")
print(f"   - Yield: {best_strategy['Yield (%)']:.2f}%")
print(f"   - Total Payoff: {best_strategy['Total Payoff']:.2f}")
print(f"   - Cooperation Rate: {best_strategy['Cooperate Rate']:.2%}")

print(f"\n2. WORST PERFORMING STRATEGY:")
print(f"   - Strategy: {worst_strategy['Strategy']} (Binary: {worst_strategy['Binary']})")
print(f"   - Yield: {worst_strategy['Yield (%)']:.2f}%")
print(f"   - Total Payoff: {worst_strategy['Total Payoff']:.2f}")
print(f"   - Cooperation Rate: {worst_strategy['Cooperate Rate']:.2%}")

print(f"\n3. CORRELATION INSIGHTS:")
correlation = summary_df['Cooperate Rate'].corr(summary_df['Yield (%)'])
print(f"   - Correlation between Cooperation Rate and Yield: {correlation:.3f}")

if correlation > 0.5:
    print(f"   - Strong positive correlation: More cooperation tends to lead to better yields")
elif correlation < -0.5:
    print(f"   - Strong negative correlation: More defection tends to lead to better yields")
else:
    print(f"   - Weak correlation: Cooperation rate has limited impact on yields")

print("\n" + "="*80)

## Conclusion

This experiment demonstrates how different prisoner dilemma strategies perform when applied to stock market data. The strategies that adapt to market conditions (price movements) show varying levels of success, providing insights into game theory applications in financial contexts.

Key findings:
- Different strategies yield significantly different results
- The cooperation rate and strategy pattern both influence performance
- Some strategies are more robust to market volatility than others